In [3]:
## import data into a ndarray 
import gzip
import numpy as np 
import random
import math 
from scipy.special import expit
f = gzip.open('zip.train.gz', 'rb')
trainData = np.genfromtxt(f)
f.close()
f = gzip.open('zip.test.gz', 'rb')
testData = np.genfromtxt(f)
f.close()


In [4]:
## separate the sevens and nines from the training data
X = trainData[(trainData[:,0] == 9) |(trainData[:,0] == 7) , :]
## Y_train = trainData[(trainData[:,0] == 9) |(trainData[:,0] == 7) , 0] - 7
## set targets for 7->0 and 9->1 
## Y_train = [1 if x == 2 else 0 for x in Y_train]
## separate the sevens and nines from the test data
Y_test = testData[(testData[:,0] == 9) |(testData[:,0] == 7) , 0] - 7 
Y_test = [1 if x == 2 else 0 for x in Y_test]
X_test = testData[(testData[:,0] == 9) |(testData[:,0] == 7) , 1:]

In [27]:
## get the targets of the randomly selected training data 
Y_train = X[:, 0] - 7
## set targets for 7->0 and 9->1 
Y_train = [1 if x == 2 else 0 for x in Y_train]
X_train = X[:,1:]

## reduce dimensionality down to 40 principle components 
from sklearn.decomposition import PCA
pca = PCA(n_components = 40 )
#train_7 = pca.fit_transform(sevens)
#train_9 = pca.fit_transform(nines)
train = pca.fit_transform(X_train)

sevens = list([train[i] for i in range(len(X)) if X[i][0] == 7])
nines = list([train[i] for i in range(len(X)) if X[i][0] == 9])

In [57]:
len(nines)

644

In [66]:
tree = dTree()
tree.create(sevens, nines, train, Y_train)
prediction = tree.predict(train[0])
print prediction

this is the total shape: (1289, 40)
New Class 1: (right and wrong)
526 245
New Class 2: (right and wrong)
399 119
Do I branch here? (left)
this is the total shape: (771, 40)
New Class 1: (right and wrong)
526 245
New Class 2: (right and wrong)
0 0
Do I branch here? (left)
this is the total shape: (771, 40)
New Class 1: (right and wrong)
492 229
New Class 2: (right and wrong)
16 34
Do I branch here? (left)
this is the total shape: (721, 40)
New Class 1: (right and wrong)
492 229
New Class 2: (right and wrong)
0 0
Do I branch here? (left)
this is the total shape: (721, 40)
New Class 1: (right and wrong)
492 229
New Class 2: (right and wrong)
0 0
Do I branch here? (left)
this is the total shape: (721, 40)
New Class 1: (right and wrong)
492 229
New Class 2: (right and wrong)
0 0
Do I branch here? (left)
this is the total shape: (721, 40)
New Class 1: (right and wrong)
492 229
New Class 2: (right and wrong)
0 0
Do I branch here? (left)
this is the total shape: (721, 40)
New Class 1: (right 

NameError: global name 'new_class2_wrong' is not defined

In [ ]:
### def getData(): 
    ## randomly sample 2/3'rds of the data 
    np.random.shuffle(X)
    sampleSize = 2*len(X)/3
    X_train = X[0:sampleSize] 
    sevens = np.array([item[:] for item in X_train if item[0]== 7], dtype=np.float)
    nines = np.array([item[:] for item in X_train if item[0]== 9], dtype=np.float)
 
    ## get the targets of the randomly selected training data 
    Y_train = X_train[:, 0] - 7
    ## set targets for 7->0 and 9->1 
    Y_train = [1 if x == 2 else 0 for x in Y_train]
    X_train = X_train[:,1:]

    ## reduce dimensionality down to 40 principle components 
    from sklearn.decomposition import PCA
    pca = PCA(n_components = 40 )
    train_7 = pca.fit_transform(sevens)
    train_9 = pca.fit_transform(nines)
    train = pca.fit_transform(X_train)

    ## repeat for test data
    X_test_pca = pca.transform(X_test)
    test =  list(X_test_pca)
    
    return test, train, train_7, train_9, Y_train

In [17]:
from scipy import asarray as ar,exp
def gausCreate(currentNode, x, classX):
    currentNode.stdD = np.std(classX)
    currentNode.mean = np.mean(classX)
    ##1/(guas.stdD * math.sqrt(2*math.pi))*exp((-(x.T - guas.mean)**2)/(2 * guas.stdD**2))
    prob = (1/(currentNode.stdD* math.sqrt(2*math.pi)))*exp((-(x-currentNode.mean)**2)/(2*currentNode.stdD**2)) 
    return prob

def gausPredict(x, gaus):
    prob = 1/(gaus.stdD * math.sqrt(2*math.pi))*exp((-(x.T - gaus.mean)**2)/(2 * gaus.stdD**2)) 
    return prob

In [65]:
class dTree: 
    def __init__(self): 
        self.root = None 
        self.current = None

    def create(self, class1, class2, total, targets): 
        self.current = self.root = treeNode(None) 
        self.makeTree(class1, class2, total, targets)
        

    def makeTree(self, class1, class2, total, targets):
        self.new_class1_right = []
        self.new_class1_wrong = []
        self.new_class2_right = []
        self.new_class2_wrong = []
        self.class1_targets = []
        self.class2_targets = []

        n1 = len(class1)
        n2 = len(class2)
        mean1 = np.mean(class1, axis=0)
        mean2 = np.mean(class2, axis=0)

        #calculate variance within class
        covar1 = np.cov(zip(*class1))
        covar2 = np.cov(zip(*class2)) 
        covarTotal = covar1 + covar2 
        covarInv = np.linalg.inv(covarTotal) 

        #calculate weights which maximize linear separation
        line = np.dot(covarInv, (mean1 - mean2))
        self.current.cut = line 
        class1_oneD = np.dot(class1, line) 
        class2_oneD = np.dot(class2, line)
        print "this is the total shape:", total.shape
        for i in range(0,total.shape[0]): 
            class1_pred = gausCreate(self.current.gaus1, total[i], class1_oneD)
            class2_pred = gausCreate(self.current.gaus2, total[i], class2_oneD) 
            ## if the classifier said class 1 
            if class1_pred[1] > class2_pred[1]:
                self.class1_targets.append(targets[i])
                ##sort the results of each gaussian fit 
                if (targets[i] == 0):
                    self.new_class1_right.append(total[i])
                else:    
                    self.new_class1_wrong.append(total[i])

            ## if the classifier said class 2         
            else: 
                self.class2_targets.append(targets[i])
                ## sort the results of each gaussian fit 
                if (targets[i] == 1):
                    self.new_class2_right.append(total[i])
                else:    
                    self.new_class2_wrong.append(total[i])
        print "New Class 1: (right and wrong)"
        print len(self.new_class1_right), len(self.new_class1_wrong)          
        print "New Class 2: (right and wrong)"
        print len(self.new_class2_right), len(self.new_class2_wrong)
        if (self.new_class1_wrong == []): 
            return None
        else:
            print "Do I branch here? (left)" 
            new_total_1 = np.concatenate((self.new_class1_right, self.new_class1_wrong))
            self.current.left = treeNode(None) 
            self.current = self.current.left
            self.makeTree(self.new_class1_right, self.new_class1_wrong, new_total_1, self.class1_targets)
        
        if (new_class2_wrong == []):
            return None
        else: 
            #print "Do I get here? (right)" 
            new_total_2 = np.concatenate((self.new_class2_right, self.new_class2_wrong))
            self.current.right = treeNode(None)
            self.current = self.current.right
            self.makeTree(self.new_class2_right, self.new_class2_wrong, new_total_2, self.class2_targets)

    def predict(self, data):
        self.current = self.root
        return self.predict_(data)
        
    def predict_(self, data): 
        print self.current.cut
        projection = np.dot(data, self.current.cut) 
        class1_pred = gausPredict(projection, self.current.gaus1)
        class2_pred = gausPredict(projection, self.current.gaus2) 
        ## if the classifier said class 1 
        if class1_pred > class2_pred:
            if self.current.left != None:
                self.current = self.current.left
                return self.predict_(data)
            else:    
                return 0
        else: 
            if self.current.right != None: 
                self.current = self.current.right
                return self.predict_(data) 
            else:
                return 1   
            
        
class treeNode: 
    def __init__(self, cut, left = None, right = None):
        self.cut = cut
        self.gaus1 = gaus()
        self.gaus2 = gaus()
        self.left = left 
        self.right = right

class gaus: 
    def __init__(self): 
        self.mean = None
        self.stdD = None

In [12]:
def main(): 
    forest = []
    guesses = np.zeros((X_test.shape[0],2)) 
    guesses_train = np.zeros((X.shape[0],2)) 
    yhat = np.zeros(X_test.shape[0]) 
    for i in range(1, 10): 
        tree = dTree()
        test, total, class1, class2, targets = getData()    
        tree.create(class1, class2, total, targets)
        forest.append(tree) 
    for currentTree in forest:
        i = 0 
        for item in total: ## this is assuming that the test data works, you'll most likely need to change that! 
            prediction = currentTree.predict(item)
            guesses_train[i][prediction] += 1
            i += 1
    yhat_train = np.zeros(total.shape[0]) 
    for i in range(len(targets)): 
        yhat_train[i] = np.argmax(guesses_train[i])
    
    for currentTree in forest:
        i = 0 
        for item in test: ## this is assuming that the test data works, you'll most likely need to change that! 
            prediction = currentTree.predict(item)
            guesses[i][prediction] += 1
            i += 1
    for i in range(len(yhat)): 
        yhat[i] = np.argmax(guesses[i])
        
    return targets, yhat_train, yhat

In [38]:
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
%matplotlib inline

tree = dTree()
 

## get the targets of the randomly selected training data 
Y_train = X[:, 0] - 7
## set targets for 7->0 and 9->1 
Y_train = [1 if x == 2 else 0 for x in Y_train]
X_train = X[:,1:]

## reduce dimensionality down to 40 principle components 
from sklearn.decomposition import PCA
pca = PCA(n_components = 40 )
#train_7 = pca.fit_transform(sevens)
#train_9 = pca.fit_transform(nines)
train = pca.fit_transform(X_train)

sevens = np.array([train[i] for i in range(len(X)) if X[i][0] == 7], dtype=np.float)
nines = np.array([train[i] for i in range(len(X)) if X[i][0] == 9], dtype=np.float)
## repeat for test data
X_test_pca = pca.transform(X_test)


##test, total, class1, class2, targets = getData() 
guesses_train = np.zeros((total.shape[0],2))
yhat_train = np.zeros(total.shape[0]) 

tree.create(class1, class2, total, targets)
#i = 0 
#for item in total: 
prediction = tree.predict(total[0])
    #guesses_train[i][prediction] += 1
    #i += 1
#for i in range(len(targets)): 
 #   yhat_train[i] = np.argmax(guesses_train[i])

#cm1 = confusion_matrix(targets, yhat_train)
# Show confusion matrix in a separate window
#print "Training Data: "
#plt.matshow(cm1)
#plt.title('Confusion matrix')
#plt.colorbar()
#plt.ylabel('True label')
#plt.xlabel('Predicted label')
#plt.show()

[ -7.23429219e-19   1.55080135e-18  -2.47753434e-17  -4.36232384e-17
  -7.84859818e-17   6.75316864e-18  -6.02871547e-17  -4.80985169e-17
  -1.30650810e-16  -1.92212181e-17  -2.84150047e-17  -8.72607764e-18
  -8.55672743e-17   4.86740183e-17   1.23694511e-17  -3.67143582e-16
   9.16560857e-17  -3.77356927e-16  -2.55232182e-16  -4.16450594e-16
   1.32481107e-16   3.24979352e-16  -9.28685997e-16   5.21248031e-16
  -1.13613484e-15  -1.06010151e-16  -3.38451384e-16  -5.41769995e-16
  -1.18352971e-15   4.58439597e-16  -1.09471834e-15   8.81757290e-16
  -4.91243384e-17  -2.61541659e-16   3.01448993e-17   5.04279041e-16
  -2.75755249e-16   2.87722727e-16  -8.80041330e-16   3.03656371e-16]


In [30]:
print targets[0:10], yhat_train[0:10]

 [1, 1, 0, 1, 0, 1, 1, 1, 1, 1] [ 0.  1.  0.  0.  0.  1.  0.  1.  0.  0.]


In [9]:
print targets[0:10], yhat_train[0:10]

[0, 0, 1, 0, 1, 0, 1, 1, 0, 1] [ 0.  0.  0.  0.  0.  1.  0.  0.  0.  1.]


In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix

##targets, yhat_train, yhat = main()
cm2 = confusion_matrix(Y_test, yhat)    
cm1 = confusion_matrix(targets, yhat_train)
# Show confusion matrix in a separate window
print "Training Data: "
plt.matshow(cm1)
plt.title('Confusion matrix')
plt.colorbar()
plt.ylabel('True label')
plt.xlabel('Predicted label')
plt.show()

# Show confusion matrix in a separate window
print "Test Data: "
plt.matshow(cm2)
plt.title('Confusion matrix')
plt.colorbar()
plt.ylabel('True label')
plt.xlabel('Predicted label')
plt.show()
